In [14]:
import numpy as np

In [29]:
UP  = 0
DOWN = 1
RIGHT = 2
LEFT = 3

def create_grid_world(size=4):
    def get_next(s,direction):
        if s == 0:
            return 0
        if direction == UP:
            return s - size if s >= size else s
        if direction == DOWN:
            return (s + size) % (size * size - 1) if s < size * (size - 1) else s
        if direction == RIGHT:
            return (s + 1) % (size * size - 1) if ((s + 1) % size) != 0  else s
        if direction == LEFT: 
            return s - 1 if (s % size) != 0 else s
            
    
    states = list(range(size*size - 1))
    actions = [UP,DOWN,RIGHT, LEFT]
    rewards = [-1]

    dynamic = {}
    for state in states:
        dynamic[state] = {}
        for action in actions:
            next_state = get_next(state, action)
            dynamic[state][action] = [(1,(next_state, -1 if state != 0 else 0))]
        
    return states, actions, rewards, dynamic
    

In [27]:
states, actions,rewards, dynamic = create_grid_world()

In [28]:
dynamic

{0: {0: [(1, (0, 0))], 1: [(1, (0, 0))], 2: [(1, (0, 0))], 3: [(1, (0, 0))]},
 1: {0: [(1, (1, -1))],
  1: [(1, (5, -1))],
  2: [(1, (2, -1))],
  3: [(1, (0, -1))]},
 2: {0: [(1, (2, -1))],
  1: [(1, (6, -1))],
  2: [(1, (3, -1))],
  3: [(1, (1, -1))]},
 3: {0: [(1, (3, -1))],
  1: [(1, (7, -1))],
  2: [(1, (3, -1))],
  3: [(1, (2, -1))]},
 4: {0: [(1, (0, -1))],
  1: [(1, (8, -1))],
  2: [(1, (5, -1))],
  3: [(1, (4, -1))]},
 5: {0: [(1, (1, -1))],
  1: [(1, (9, -1))],
  2: [(1, (6, -1))],
  3: [(1, (4, -1))]},
 6: {0: [(1, (2, -1))],
  1: [(1, (10, -1))],
  2: [(1, (7, -1))],
  3: [(1, (5, -1))]},
 7: {0: [(1, (3, -1))],
  1: [(1, (11, -1))],
  2: [(1, (7, -1))],
  3: [(1, (6, -1))]},
 8: {0: [(1, (4, -1))],
  1: [(1, (12, -1))],
  2: [(1, (9, -1))],
  3: [(1, (8, -1))]},
 9: {0: [(1, (5, -1))],
  1: [(1, (13, -1))],
  2: [(1, (10, -1))],
  3: [(1, (8, -1))]},
 10: {0: [(1, (6, -1))],
  1: [(1, (14, -1))],
  2: [(1, (11, -1))],
  3: [(1, (9, -1))]},
 11: {0: [(1, (7, -1))],
  1: [(1,

In [22]:
random_policy = {
    state: [0.25,0.25,0.25,0.25] for state in states}

In [89]:
def policy_evaluation(states, policy, dynamic, actions, theta, discount =  1, max_iter = 100, V_init = None):
    if V_init is None:
        V = np.zeros(len(states))
    else: 
        V = V_init[:]
        
    delta = 0
    k = 0
    while k < max_iter:
        for state in states:
            if state != 0:
                v  = V[state]
                partial_action = 0
                for action in actions:
                    p_action = policy[state][action]
                    transitions = dynamic[state][action]
                    partial_transitions = 0
                    for transition in transitions:
                        p_transition, (next_state, reward) = transition
                        partial_transitions += p_transition *  (reward + discount * V[next_state])
                    partial_action += p_action * partial_transitions
                V[state] = partial_action

                delta = max(delta, abs(v - V[state])) 

        if delta < theta:
            return V
        k += 1
    return V

In [76]:
V = policy_evaluation(states, random_policy,dynamic, actions, 1,1, max_iter=1000)

In [90]:
def greedy_policy(states, actions, policy, dynamic, discount):
    V = policy_evaluation(states, policy,dynamic, actions, 0.5,discount, max_iter=1000)

    greedy = {0: [1] + [0] * (len(actions) - 1)}
    for state in states : 
        if state != 0:
            best_action  = actions[0]
            transitions = dynamic[state][best_action]
            best_q = 0
            greedy[state] = [0] * len(actions)
            for transition in transitions:
                p_transition, (next_state, reward) = transition
                best_q += p_transition *  (reward + discount * V[next_state])

            for action in actions[1:]:
                q = 0
                transitions = dynamic[state][action]
                for transition in transitions:
                    p_transition, (next_state, reward) = transition
                    q += p_transition *  (reward + discount * V[next_state])
                if q > best_q:
                    best_action = action
                    best_q = q
            greedy[state][best_action] = 1

    return greedy

In [91]:
greedy = greedy_policy(states, actions, random_policy, dynamic,1)

In [92]:
V

array([  0., -14., -20., -22., -14., -18., -20., -20., -20., -20., -18.,
       -14., -22., -20., -14.])

In [93]:
greedy

{0: [1, 0, 0, 0],
 1: [0, 0, 0, 1],
 2: [0, 0, 0, 1],
 3: [0, 1, 0, 0],
 4: [1, 0, 0, 0],
 5: [0, 0, 0, 1],
 6: [0, 1, 0, 0],
 7: [0, 1, 0, 0],
 8: [1, 0, 0, 0],
 9: [1, 0, 0, 0],
 10: [0, 0, 1, 0],
 11: [0, 1, 0, 0],
 12: [1, 0, 0, 0],
 13: [0, 0, 1, 0],
 14: [0, 0, 1, 0]}

In [95]:
def policy_iteration(states, actions, dynamic, discount = 1, V_init=None, max_iter = 100):
    # Initialization 
    if V_init is None:
        V = np.zeros(len(states))
    else:
        V = V_init[:]

    policy = {s : [1] + [0] * (len(actions) - 1) for s in states}

    for _ in range(max_iter):
        # policy evaluation 
        V = policy_evaluation(states, policy, dynamic, actions, 0.1,discount, V_init=V)
    
        # policy improvement
        stable = True
        new_policy = greedy_policy(states, actions, policy, dynamic, discount)
        for state in states:
            old_action = np.argmax(policy[state])
            new_action = np.argmax(new_policy[state])
            if old_action != new_action:
                stable = False
                break
        if stable:
            return new_policy
        policy = new_policy
    return policy

In [96]:
policy = policy_iteration(states, actions, dynamic)

In [98]:
V = policy_evaluation(states, policy,dynamic, actions, 1,1, max_iter=1000)

In [99]:
V

array([ 0., -1., -2., -3., -1., -2., -3., -2., -2., -3., -2., -1., -3.,
       -2., -1.])